In [1]:
import pandas as pd
import numpy as np
import os,glob
from matplotlib import pyplot as plt

# run a random forest on these words and see how well it can identify labels

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
glob.glob('third*.json')

['third_encoded_LDA_with_topics.json']

In [3]:
df = pd.read_json('third_encoded_LDA_with_topics.json')

In [6]:
#get the feature columns: those that start with 'key_word:'
feat = [x for x in df.columns if x[:9] == 'key_word:']
len(feat)

13272

# choose two topics to classify

In [46]:
topics = ['hope','home']
sizes = []
for topic in topics:
    temp = df.loc[df[topic] == 1].shape[0]
    sizes.append(temp)
    print('the topic',topic,'has',temp,'talks associated')

the topic hope has 42 talks associated
the topic home has 77 talks associated


In [27]:
# make an even dataset
one = df.loc[df[topics[0]] == 1].iloc[:np.array(sizes).min()].copy()
two = df.loc[df[topics[0]] == 1].iloc[:np.array(sizes).min()].copy()
print(one.shape[0],two.shape[0])
new = pd.concat([one,two])

42 42


In [16]:
new.shape

(84, 13583)

# are there any instances that are both topics

In [25]:
remove_ind = list(set(new.loc[(new[topics[0]] == 1) & (new[topics[1]] == 1)].index))
if len(remove_ind) > 0:
    print(len(remove_ind))
    new.drop(index=remove_ind,inplace=True)

# reduce the feature to where only these talks have these words so there is a one in each column

In [36]:
# get all the features 
b = new
all_key_words = set()
for i in range(b.shape[0]):
    all_key_words = all_key_words.union(set(b.iloc[i]['words']))
new_feat = ['key_word:' + word for word in all_key_words]
print(len(new_feat))

1738


# PREDICT WHETHER THE TALK IS HOME

# test train split

In [38]:
X = new[new_feat].copy()
print('the target variable is:',topics[0])
y = new[topics[0]]
print('X.shape',X.shape)

the target variable is: hope
X.shape (84, 1738)


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)


In [54]:
for obj in [X_train, X_test, y_train, y_test]:
    print(obj.shape)

(75, 1738)
(9, 1738)
(75,)
(9,)


In [55]:
# X_train

# RUN THE MODEL 

In [56]:
clf = RandomForestClassifier(n_estimators=10
                             ,n_jobs=-1
                             ,oob_score=True)
clf = clf.fit(X_train, y_train)

/Users/joeywilkes/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/joeywilkes/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [57]:
#mean accuracy 
clf.score(X_test,y_test)

1.0

In [58]:
clf.predict_proba(X_test)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

# IF IT'S STRUGGLING THEN TRY NORMALIZING FEATURES